In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [4]:
df = pd.read_csv("TV.csv", encoding="utf-8")
df

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
2,LG,4.4,38870,3443,16499,21990,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: WebOS,HD Ready 1366 x 768 Pixels,10 W Speaker Output,50 Hz Refresh Rate
3,OnePlus,4.3,101256,9189,16499,21999,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: Android,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
4,Xiaomi,4.3,3120,305,15499,24999,Netflix|Prime Video|Disney+Hotstar|Youtube,Operating System: Android,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate
...,...,...,...,...,...,...,...,...,...,...,...
661,SONY,4.6,0,0,59999,94900,Ultra HD (4K) 3840 x 2160 pixels Pixels,NO Speaker Output,60 Hz Refresh Rate,2 x HDMI | 1 x USB,1 YEAR
662,SONY,4.5,6056,1082,34184,68400,Netflix|Prime Video|Youtube,Operating System: Linux based,Ultra HD (4K) 3840 x 2160 Pixels,20 W Speaker Output,50 Hz Refresh Rate
663,Croma,0.0,0,0,9194,20000,HD Ready 1366 x 786 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,3 Years Warranty
664,T-Series,0.0,0,0,26999,33990,Netflix|Prime Video|Apple TV|Disney+Hotstar|Yo...,Full HD 1920 x 1080 Pixels,16 Speaker Output,50 Hz Refresh Rate,2 x HDMI | 2 x USB


In [5]:
df.dtypes

Product_Name         object
Stars               float64
Ratings               int64
Reviews               int64
current_price         int64
MRP                   int64
channel              object
Operating_system     object
Picture_quality      object
Speaker              object
Frequency            object
dtype: object

## Q1.

In [106]:
df_freq = df.copy()

df_freq["CorrFreq"] = np.NaN
df_freq.loc[df_freq["Picture_quality"].str.contains("\d{2,3} Hz", regex=True), "CorrFreq"] = df_freq["Picture_quality"]
df_freq.loc[df_freq["Speaker"].str.contains("\d{2,3} Hz", regex=True), "CorrFreq"] = df_freq["Speaker"]
df_freq.loc[df_freq["Frequency"].str.contains("\d{2,3} Hz", regex=True), "CorrFreq"] = df_freq["Frequency"]
df_freq = df_freq.loc[df_freq["CorrFreq"].notna(), ]
df_freq.loc[df_freq["CorrFreq"].str.contains("^60 Hz")]["CorrFreq"].count()

#510

510

In [116]:
#Q1. 정답
df_q1 = df[["Frequency", "Picture_quality", "Speaker"]].copy()
ser_u = pd.Series(df_q1["Frequency"].unique())

df_q1["hz_1"] = df_q1["Frequency"].str.contains("[0-9]{2,3} Hz") + 0
df_q1["hz_2"] = df_q1["Picture_quality"].str.contains("[0-9]{2,3} Hz") + 0
df_q1["hz_3"] = df_q1["Speaker"].str.contains("[0-9]{2,3} Hz") + 0
# 위 3개가 0, 0, 0 이면 결측

결측치개수 = df_q1.loc[:, "hz_1":"hz_3"].sum(axis=1).value_counts()[0]
df_q1 = df_q1.loc[df_q1.loc[:, "hz_1":"hz_3"].sum(axis=1) != 0]
결측지제거후수 = len(df_q1)

tx_regex = "[^0-9]60 Hz|^60 Hz"
df_q1["cnt_60Hz"] = df_q1.loc[:, :"Speaker"].apply(lambda x: x.str.contains(tx_regex).sum(), axis = 1)
df_q1["cnt_60Hz"].value_counts()[1]



510

In [105]:
# Easy One
df_q1.apply(lambda x: x.str.contains("60 Hz").sum(), axis=1).value_counts()[1]

510

## Q2.

In [78]:
df_pq = df.copy()

m8k = df_pq.loc[df_pq["Picture_quality"].str.contains("8K") | df_pq["Operating_system"].str.contains("8K") | df_pq["channel"].str.contains("8K"), "Stars"].mean()
m4k = df_pq.loc[df_pq["Picture_quality"].str.contains("4K") | df_pq["Operating_system"].str.contains("4K") | df_pq["channel"].str.contains("4K"), "Stars"].mean()
diff = abs(m8k - m4k)
round(diff, 2)
#0.38

0.38

In [132]:
## Q2. 정답

df_q2 = df[["Stars", "Operating_system", "channel", "Picture_quality"]].copy()
#ser_u = pd.Series(df_q2["Operating_system"].unique())
# ser_u = pd.Series(df_q2["channel"].unique())
ser_u = pd.Series(df_q2["Picture_quality"].unique())
ser_u[ser_u.str.contains("4K|8K")]

df_q2["cnt_4k"] = df_q2.iloc[:, [1,2,3]].apply(lambda x: x.str.contains("4K").sum(), axis=1)
df_q2["cnt_8k"] = df_q2.iloc[:, [1,2,3]].apply(lambda x: x.str.contains("8K").sum(), axis=1)



stat_4k = df_q2.loc[df_q2["cnt_4k"] >= 1, "Stars"].mean()
stat_8k = df_q2.loc[df_q2["cnt_8k"] >= 1, "Stars"].mean()
round(abs(stat_4k - stat_8k), 2)


0.38

## Q3.

In [93]:
df_ev = df.copy()

df_ev["Ratio"] = df_ev["Reviews"] / df_ev["Ratings"]
df_ev["DiscountRatio"] = df_ev["current_price"] / df_ev["MRP"]
df_ev["Netflix"] = 0
df_ev.loc[df_ev["channel"].str.contains("Netflix"), "Netflix"] = 1
df_ev["PrimeVideo"] = 0
df_ev.loc[df_ev["channel"].str.contains("Prime Video"), "PrimeVideo"] = 1
df_ev["HighDefinition"] = 0
df_ev.loc[df_ev["Picture_quality"].str.contains("8K"), "HighDefinition"] = 1
df_ev.loc[df_ev["Picture_quality"].str.contains("4K"), "HighDefinition"] = 1

df_ev.head(2)

# 위에까지 하고 random forest 는 하지도 못함.

,Product_Name,Stars,Ratings,Reviews,current_price,MRP,channel,Operating_system,Picture_quality,Speaker,Frequency,Ratio,DiscountRatio,Netflix,PrimeVideo,HighDefinition
0,Croma,4.2,1773,217,7990,20000,HD Ready 1366 x 768 Pixels,20 Speaker Output,60 Hz Refresh Rate,2 x HDMI | 2 x USB,1 Year Warranty,0.122391,0.399500,0,0,0
1,Adsun,3.8,6742,930,8699,21999,Netflix|Disney+Hotstar|Youtube,Operating System: Android Based,HD Ready 1366 x 768 Pixels,20 W Speaker Output,60 Hz Refresh Rate,0.137941,0.395427,1,0,0


In [147]:
## Q3 정답
df_q3 = df.loc[~df["channel"].str.contains("Pixel|Oper"), ].reset_index(drop = True)
df_q3["x1"] = df_q3["Reviews"] / df_q3["Ratings"]
df_q3["x2"] = df_q3["MRP"]
df_q3["x3"] = df_q3["current_price"] / df_q3["MRP"]
df_q3["x4"] = df_q3["channel"].str.contains("Netflix") + 0
df_q3["x5"] = df_q3["channel"].str.contains("Prime Video") + 0
df_q3["x6"] = df_q3["Picture_quality"].str.contains("4K|8K") + 0

df_q3_sub = df_q3[["Stars", "x1", "x2", "x3", "x4", "x5", "x6"]]

df_q3_sub.isna().sum() # 351 개가 x1에 있다.

df_model = df_q3_sub.dropna()

model = RandomForestRegressor(random_state=123)
model.fit(X=df_model.drop(columns = "Stars"), y =df_model["Stars"])
model.feature_importances_ # 음 x3이 매우 중요하군

ser_f = pd.Series(model.feature_importances_, index = df_model.columns[1:])
ser_f.idxmax() # 할인율

'x3'